# Aprendizado não-supervisionado

In [ ]:
import pandas
titanic = pandas.read_csv("https://databootcamp.nyc3.digitaloceanspaces.com/titanic_3.csv")

titanic.head()

## Distâncias

Vamos analisar o conceito de distância inicialmente! Esse conceito é usado por muitos dos algoritmos de aprendizado não supervisionado.

Vamos imaginar um conjunto de dados que pode ser representado por dois valores numéricos, $X_1$ e $X_2$

Vamos ver esses dados em um gráfico com os valores indicando suas posições

In [ ]:
dados_teste = [[1,2],
                [2,1],
                [2,3.2],
                [3,2.5],
                [4,1],
                [1.2,1],
                [0.2,7.3]]

dados = pandas.DataFrame(dados_teste,columns=["X1", "X2"])
dados

Vamos colocar esses pontos em um gráfico para ilustrar como estão espalhados

In [ ]:
from matplotlib import pyplot

pyplot.plot(dados["X1"], dados["X2"], "o")

Podemos ver visualmente que existe uma distância a ser medida entre esses pontos. Podemos medir essas distâncias de algumas formas. Vamos ver 2 delas.

### Euclidiana

A distância euclidiana é a distância direta entre dois pontos no espaço. Ela é bem útil para medir uma distância direta entre um ponto e outro e será usada no algoritmo KMeans que vamos ver a seguir.

![euclidean](https://databootcamp.nyc3.digitaloceanspaces.com/img/distance.jpg)

Quão mais perto de zero, menor a distância entre os pontos.

Nesse caso, com apenas duas colunas, podemos medir a distância entre dois pontos visualmente.
A distância em linha reta desses pontos pode ser calculada a partir da fórmula da **distância euclidiana**

$D(a,b) = \sqrt{(a_{x1} - b_{x1})^2 + (a_{x2} - b_{x2})^2}$

Vamos fazer então a função que calcula a distância euclidiana

In [ ]:
def dist(a,b):
    return ( (a[0]-b[0])**2 + (a[1]-b[1])**2 )**0.5

In [ ]:
dado_a = dados_teste[0]
dado_b = dados_teste[2]
d_ab = dist(dado_a,dado_b)

print(f"A distância de {dado_a} a {dado_b} é {d_ab}")

Esse é o caso de um dado com dois atributos (duas dimensões). Podemos generalizar o cálculo para a quantidade de dimensões da coluna. A distância de duas colunas que tem $n$ atributos é

$D(a,b) = \sqrt{ \sum_{i=1}^{n}{(a_{xi} - b_{xi})^2}}$

Com isso conseguimos implementar essa generalização

In [ ]:
def dist_n(a,b):
    somatorio = 0.0
    for i in range(len(a)):
        somatorio = somatorio + (a[i]-b[i])**2
    return somatorio**0.5

Para as mesmas listas temos uma distância zerada, pois são iguais

In [ ]:
dist_n([1, 2],[1, 2])

Podemos validar a distância entre as nossas 2 colunas

In [ ]:
dist_n(dados["X1"], dados["X2"])

No **Sklearn** temos a distância euclidiana já implementada que é muito mais robusta que a nossa, mas ela espera que várias colunas sejam passadas, então precisamos passar uma lista

In [ ]:
from sklearn.metrics.pairwise import paired_distances

In [ ]:
paired_distances([dados["X1"]], [dados["X2"]])

## <font color='blue'>Encontre a distância euclidiana entre a coluna *SibSp* e *Relatives* no Titanic</font>

![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/atrasada-relogio-pulso-1116-1400x800.jpg)

In [ ]:
#solução

### Cosseno

A similaridade entre os cossenos também é muito utilizada em muitos algoritmos de Machine Learning não supervisionados. Muitos deles normalmente algoritmos de recomendação.

Diferente da distância euclidiana, essa métrica não está interessada na magnetude da distância, mas somente com a diferença dos ângulos dos vetores (que são suas colunas ou linhas).

O que vemos normalmente é a similaridade dos cossenos e a distância nada mais é do que $1-similaridade$

![similarity_cos](https://databootcamp.nyc3.digitaloceanspaces.com/img/cosine.png)

Quão mais perto de zero, menor a distância entre os pontos, enquanto que mais perto de 1, maior a distância.

Como essa distância utiliza multiplicação matricial, utilizaremos o **Numpy** para ajudar a fazer a fórmula, primeiro da similaridade

In [ ]:
import numpy

def cos_similaridade(a, b):
    dot_product = numpy.dot(a, b)
    norm_a = numpy.linalg.norm(a)
    norm_b = numpy.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

E depois a distância dos cossenos

In [ ]:
def cos_distancia(a, b):
    return 1 - cos_similaridade(a, b)

In [ ]:
cos_similaridade(dados["X1"], dados["X2"])

In [ ]:
cos_distancia(dados["X1"], dados["X2"])

No **Sklearn** temos a distância dos cossenos já implementada também com a mesma função, mas usando o argumento de métrica como "cosine". Por padrão ele usa a euclidiana.

In [ ]:
paired_distances([dados["X1"]], [dados["X2"]], "cosine")

## <font color='blue'>Encontre a distância dos cossenos entre a coluna *SibSp* e *Relatives* no Titanic</font>

![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/atrasada-relogio-pulso-1116-1400x800.jpg)

In [ ]:
#solução

## K-means

K-means é um algoritmo de agrupamento. A idéia é agrupar os dados por **proximidade** a um ponto central. Para falarmos sobre **proximidade** vamos aplicar os conceitos de **distância e similaridade** que vimos previamente. Ele usa um conceito de centróide que precisamos ver rapidamente.


### Centróide de um grupo

**Centróide** é um ponto calculado que representa o centro do conjunto. 

Um cálculo normal de centroide a média dos atributos. 

Para entender melhor, vamos calcular o centroid dos dados de teste $X_1$ e $X_2$.

In [ ]:
dados

Vamos usar o Método `mean()` da coluna para encontrar a média dos valores

In [ ]:
centroid_x1 = [dados["X1"].mean()]
centroid_x2 = [dados["X2"].mean()]
print("""
O centroid dos dados é [{},{}]
""".format(centroid_x1,centroid_x2))

Vamos plotar agora o grafico dos pontos para observar o posicionamento do centroid

In [ ]:
pyplot.plot(dados["X1"],dados["X2"],"ob")
pyplot.plot(centroid_x1, centroid_x2, "+r")

Parece realmente representar o centro do conjunto

## <font color='blue'>Faça uma função que calcule o centróide de todas as colunas do Titanic</font>
![alt text](https://static.vix.com/pt/sites/default/files/styles/large/public/a/atrasada-relogio-pulso-1116-1400x800.jpg?itok=qv3gUH6U)

In [ ]:
#solução
def centroide(df):
    pass

In [ ]:
centroide(titanic)

Agora que sabemos encontrar o centróide de um conjunto, vamos ao conceito do algoritmo de `KMeans`!

### O algortimo

O objetivo do algoritmo é : Encontrar K clusters (agrupamentos) dos dados.

Passo-a-passo: 

    1. Iniciamos K pontos aleatórios que serão considerados os centróides dos grupos.
    
    2. A matriz de distâncias é montada. 
    
  Supondo que queremos montar `K` grupos e temos `n` dados 
    
|       | $C_1$      | $C_2$      | ... |   $C_k$    |
|-------|------------|------------|-----|------------|
| $u_1$ |d($u_1$,$C_1$)|d($u_1$,$C_2$)| ... |d($u_1$,$C_k$)|
| $u_2$ |d($u_2$,$C_1$)|d($u_2$,$C_2$)| ... |d($u_2$,$C_k$)|
|  ...  |    ...     |     ...    | ... |     ...    |
| $u_n$ |d($u_n$,$C_1$)|d($u_n$,$C_2$)| ... |d($u_n$,$C_k$)|

   Cada dado $u_i$ é considerado do cluster com menor distância do centróide
    
    3. O Centróide real dos dados dos grupos é calculado
   
    4. A matriz de distância é feita de novo, considerando os centroides calculados no passo 3.
       
       4.1 . Se algum dado mudar de cluster, o algoritmo volta ao passo 2, considerando esses centroides como os K centroides 
       
       4.2 . Se nenhum dado mudar de cluster, o algoritmo assume que encontrou os clusters!


![kmeans](https://databootcamp.nyc3.digitaloceanspaces.com/img/K-means_convergence.gif)

Vamos agora criar os cluster dos dados fictícios 

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
model = kmeans.fit(dados)

Vamos agora pegar os centróides do modelo e plotar. O **T** é usado para transpor nosso conjunto de dados para facilitar o plot

In [ ]:
centers = model.cluster_centers_
c_to_plot = centers.T

In [ ]:
pyplot.plot(dados["X1"],dados["X2"],'ob')
pyplot.plot(c_to_plot[0],c_to_plot[1],'or')

### Agora que já vimos como o KMeans funciona, vamos aplicar para um problema real!

Vamos tentar verificar se podemos clusterizar os tripulantes do Titanic escolhendo algumas Features

In [ ]:
features = [
        "Pclass",
        "Sex",
        "SibSp",
        "Parch",
        "Relatives",
        "Embarked",
        "Survived",
        "Fare"
    ]
feature_df = pandas.get_dummies(titanic[features])
feature_df.head()

Como vamos precisar do Scaler também. Precisamos do Scaler porque a medida de distância é muito afetada se não padronizarmos os números.

In [ ]:
from sklearn.preprocessing import StandardScaler

Vamos começar com 4 clusters igual fizemos anteriormente com: criança, adolescente, adulto, idoso

In [ ]:
scaler = StandardScaler()
kmeans = KMeans(n_clusters=4, random_state=42)

In [ ]:
scaled_df = scaler.fit_transform(feature_df)
predictions = kmeans.fit_predict(scaled_df)
predictions.shape

Com isso conseguimos os Clusters que o algoritmo predice

In [ ]:
predictions

Mas será que isso está certo? Vamos avaliar

## Métricas

Precisamos saber metrificar os algoritmos não supervisionados também. Como não temos um parâmetro de verdade, as métricas usam outros parâmetros

### Inércia

A inércia é a métrica mais conhecida de clusters. Ela mede o quão distante os pontos estão do centróides em um cluster. Quão mais próximo de zero, melhor é sua clusterização.

Para avaliar isso, precisamos olhar no resultado do KMeans.

In [ ]:
kmeans.inertia_

Esse valor parece meio alto... Será que realmente é a melhor quantidade de clusters? Para isso temos o método de **Elbow**.

### Elbow

Método que visa encontrar a melhor quantidade de clusters para um problema. Com isso você pode avaliar se o seu número de clusters realmente era bom. A idea dele é simplesmente iterar sobre vários números possíveis de cluster e avaliar se o valor de **Inércia** cai. A melhor quantidade de clusters normalmente é aonde aparece o "cotovelo" (Elbow) do gráfico.

![elbow](https://databootcamp.nyc3.digitaloceanspaces.com/img/elbow.png)



In [ ]:
inercias = []
clusters = range(2, 50)
for k in clusters:
    scaler = StandardScaler()
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit_predict(
        scaler.fit_transform(feature_df)
    )
    inercias.append(kmeans.inertia_)



Tendo calculado os valores de Inércia podemos plotar e descobrir um bom cotovelo (Elbow)

In [ ]:
pyplot.plot(clusters, inercias, "o")
pyplot.grid(True)
pyplot.title('Elbow')

Normalmente esse método é bem subjetivo mesmo e podemos ver um bom valor entre 10 e 20 clusters se formos tentar dividir os tripulantes. Mas está bastante subjetivo... Um método mais objetivo é o da Silhueta (**Silhouette**)

### Silhouette

A Silhueta mede o quanto valores de um cluster estão distantes do outro cluster
O valor de Silhueta tem máximo e mínimo, sendo -1 o pior e 1 o melhor. Com isso, conseguimos entender melhor se um resultado é bom ou não.

Na fórmula, *a* é a média da distância dentro de um cluster dos outros pontos no mesmo cluster e *b* é a menor distância entre um ponto de um cluster de qualquer outro cluster.

$S(i) = (b(i) - a(i)) / max(a(i), b(i))$

Isso é feito para cada ponto *i*.

Podemos tentar encontrar o melhor número de clusters usando essa métrica para validar dentro do intervalo que encontramos em **Elbow**

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
silhuetas = []
clusters = range(10, 21)
for k in clusters:
    scaler = StandardScaler()
    kmeans = KMeans(n_clusters=k, random_state=42)
    predicted = kmeans.fit_predict(
        scaler.fit_transform(feature_df)
    )
    score = silhouette_score(feature_df, predicted)
    silhuetas.append(score)



In [ ]:
pyplot.plot(clusters, silhuetas)
pyplot.grid(True)
pyplot.title('Silhouette and Inertia')

O melhor valor parece ser realmente com 14 clusters! Mas mesmo assim ainda está bem ruim

## <font color='blue'>Vamos usar KMeans para ver quem sobrevive ou não no Titanic!</font>

Vamos tentar usar 2 clusters e prever se vive ou não... Será que funciona?
![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/atrasada-relogio-pulso-1116-1400x800.jpg)

In [ ]:
#solução